In [1]:
from simulated_annealing import *

In [2]:
def basic_example(example):
    T = 100
    r = 0.99
    tc = 1000
    hc = 1000
    solution = simulated_annealing(read_data(example), T, r/100, tc, hc)
    solution

In [ ]:
T_max = 100
r = 0.99
termination_condition = 1000
halting_condition = 1000000
test_SA(T_max, r, termination_condition, halting_condition)

rws_instances/Example1.txt
Not satified in the given time. Best solution found has a score of: 35
Last solution: 25
passed
rws_instances/Example2.txt
Not satified in the given time. Best solution found has a score of: 40
Last solution: 10
passed
rws_instances/Example3.txt
Not satified in the given time. Best solution found has a score of: 25
Last solution: 25
passed
rws_instances/Example4.txt
Not satified in the given time. Best solution found has a score of: 90
Last solution: 90
passed
rws_instances/Example5.txt


In [ ]:
solutions = run_SA_on_examples()

In [ ]:
pprint(read_data(get_examples()[0]))
print('------------------------')
pprint(read_data(get_examples()[11]))


In [ ]:
basic_example(get_examples()[11])